# Kepler's First Law
<i>The orbit of a planet is an ellipse with the Sun at one of the 2 foci</i>

This is the first of 3 Kepler's laws of planetary motion. Technically the Sun is not exactly the center of universe, it also revolves around a common center of gravity. Sun contains over 99% of the Solar System's mass. So it is possible movement of the so-called <b>Solar System Barycentre (SSB)</b> with respect to Sun's centre, but is it that significant? 

## Preparation
Last time we learned about SPICE and its way to compute miscellaneous parameters using so called kernels. We loaded the necessary kernels using the SPICE function furnsh. Each kernel was loaded individually. However, larger projects may require to load dozens of kernels. That would bloat our Python code and one could easily lose track of one’s code. To prevent this, SPICE introduced the kernel meta files. A meta file contains the relative or absolute paths to the needed kernels. For this tutorial, each part has its own meta file that sets the relative path to the _kernels directory.

The first 6 lines start with the command \begintext. The following text block is a comment section, where one can add up any project relevant information. The last block starts with \begindata. This indicates SPICE that all following parts are relevant for the coding part. KERNELS_TO_LOAD lists all kernel files that are used in this tutorial part. <b>Please note</b>: <u>If you load kernels that contain similar information (e.g., by accident you load first the naif0012.tls leap seconds kernel and afterwards the naif0011.tls kernel), SPICE considers the order and takes the last one.</u>

In [1]:
import numpy as np
import spiceypy as sp
import datetime as dt

#Load the SPICE kernels via a meta file
# sp.furnsh('kernel_meta2.txt')
sp.furnsh('_kernels/lsk/naif0012.tls')

In [2]:
sp.furnsh('_kernels/spk/de440.bsp')
sp.furnsh('_kernels/pck/pck00010.tpc')

For the position computation we need to set a starting/initial and ending time-stamp. Here we set the date 2000-01-01 (midnight) UTC and add up 10,000 days (Around 27.5 years; this value is chosen at random). These computations can be done by Python library datetime. Afterwards, the initial and ending times are converted to strings. Finally UTC strings are converted to the corresponding Ephemeris times.

We want to compute the <b>Solar System barycenter (SSB)</b> w.r.t the centre of the Sun for a certain time interval.

In [8]:
# we want to compute the Solar System barycenter (SSB) w.r.t the centre of the Sun for a certain time interval.
# Firse we set an initial time in UTC.
INIT_TIME_UTC = dt.datetime(year=2000, month=1, day=1, hour=0, minute=0, second=0)

# Add the number of days we can play around with the datetime variables
DELTA_DAYS = 10000
END_TIME_UTC = INIT_TIME_UTC + dt.timedelta(days=DELTA_DAYS)

# convert the datetime objects now to strings
INIT_TIME_UTC_STR = INIT_TIME_UTC.strftime('%Y-%m-%dT%H:%M:%S')
END_TIME_UTC_STR = END_TIME_UTC.strftime('%Y-%m-%dT%H:%M:%S')

# print the starting and end times
print('Init time in UTC: %s' % INIT_TIME_UTC_STR)
print('End time in UTC: %s\n' % END_TIME_UTC_STR)

# Convert to Ephemeris Time (ET) using the spice function utc2et
INIT_TIME_ET = sp.utc2et(INIT_TIME_UTC_STR)
END_TIME_ET = sp.utc2et(END_TIME_UTC_STR)

print('Init time in ET: %s' % INIT_TIME_ET)
print('End time in ET: %s\n' % END_TIME_ET)


Init time in UTC: 2000-01-01T00:00:00
End time in UTC: 2027-05-19T00:00:00

Init time in ET: -43135.816087188054
End time in ET: 863956869.1851972



Why are <b>leap-seconds kernels</b> necessary? A day has 24 hours, a year has 365 days and every 4 years another day is added. This applied to our everyday life. From a detailed scientific point of view our Earth revolves around itself and around the Sun in <b>slightly varying times</b> that need to be compensated. How much time is added in this 10,000 days interval starting at the millennium?

A day has <b>86,400 seconds</b>. This value times 10,000 days leads to <b>864,000,000 seconds</b>. Subtracting the initial time in ET from the ending time in ET leads to <b>864,000,005.0012845 seconds</b>. Approximately 5 seconds have been added. This might not be crutial right now, but for some high precision scientific fields every milisecond counts.

We finally set a NumPy array that contains 10,000 time steps between the starting and ending times.

A day has 86400 seconds (24 hours * 60 minutes * 60 seconds)
In our example, we set a time period of 10000 days. Thus, we expect the difference between INIT_TIME_ET and END_TIME_ET to be 10000*86400 seconds. Let's have a look at the delta.

In [4]:
# A day has 86400 seconds (24 hours * 60 minutes * 60 seconds)
# In our example, we set a time period of 10000 days. Thus, we expect the
# difference between INIT_TIME_ET and END_TIME_ET to be 10000*86400 seconds
# Let's have a look at the delta.

print('Covered time interval in seconds: %s\n' % (END_TIME_ET - INIT_TIME_ET))
# we see that approx 5 seconds are added in this (leapseconds)

Covered time interval in seconds: 864000005.0012845



In [5]:
# Create a numpy array that covers a time interval in delta=1 day step
TIME_INTERVAL_ET = np.linspace(INIT_TIME_ET, END_TIME_ET, DELTA_DAYS)


In [6]:
len(TIME_INTERVAL_ET)

10000

Now we are ready yo compute the Solar System Barycentre (SSB) position in x,y,z direction with respect to the Sun's centre. For this purpose, we compute the position using SPICE function <b>spkgps</b> (in the last tutorial we used a similar function <u><i>spkgeo</i></u> that returns the state vector (position and velocity)).
<br>

This function requires the following input parameters:

1. targ: The NAIF ID of the <b>SSB (0)</b>. The sophisticated SPICE documentation lists all the available ID codes of the Solar System as well as spacecraft missions.
2. et: The Ephermeris time
3. ref: The reference frame of interest. In our case we compute the position in <b>ECLIPJ2000</b>. So, the ecliptic plane of our home planet is the reference plane.
3. obs: The NAIF ID of the <b>Sun (10)</b>.

<i><b>spkgps</b></i> returns the position of the SSB w.r.t the Sun at a certain Ephemeris time computed in the ECLIPJ2000 reference frame. The second output value is the so called light time and stores the travelling time of the light between the Sun's centre and SSB. Since we do not need this value, we use a single underscore.

In [9]:
# Now we compute the position of the Solar System's barycentre w.r.t our Sun
# First we set an empty list that stores later all x , y, z components
# for each time step

SSB_WRT_SUN_POSITION = []

# each time step is used in this for loop to compute the position of the 
# SSB w.r.t the Sun. We use the function spkgps
for TIME_INTERVAL_ET_f in TIME_INTERVAL_ET:
    _position, _ = sp.spkgps(targ=0, et=TIME_INTERVAL_ET_f, ref='ECLIPJ2000', obs=10)
    
    # Append the result to the final list
    SSB_WRT_SUN_POSITION.append(_position)

# convert the list to a numpy array
SSB_WRT_SUN_POSITION = np.array(SSB_WRT_SUN_POSITION)

Print the result at the initial time point to get a feeling of the x,y,z components and the distance

In [12]:
SSB_WRT_SUN_POSITION[0]

array([1068108.35424524,  417721.9144422 ,  -30868.85962713])

In [11]:
len(SSB_WRT_SUN_POSITION)

10000

In [13]:
print('Position (components) of the Solar System Barycentre w.r.t the\n center of the Sun (at initial time): \n X = %s km\n Y = %s km\n Z = %s km\n' % tuple(np.round(SSB_WRT_SUN_POSITION[0])))

# lets determine and print the corresponding distance using the numpy function linalg.norm()
print('Distance between the Solar System Barycentre w.r.t the\n center of the Sun (at initial time): \n d = %s km\n' % round(np.linalg.norm(SSB_WRT_SUN_POSITION[0])))

Position (components) of the Solar System Barycentre w.r.t the
 center of the Sun (at initial time): 
 X = 1068108.0 km
 Y = 417722.0 km
 Z = -30869.0 km

Distance between the Solar System Barycentre w.r.t the
 center of the Sun (at initial time): 
 d = 1147301 km



Wedid get some large numbers. A distance betwee the Sun's centre and the SSB of over 1 million km ( around 3 times the distance between the Earth and the Moon).

Comprehending the large numbers in astronomy leads mostly to scaling, like eg 1 Astronomical Unit (AU) or 1 Light year. <u>Here we can define our own scale by using the radius of the Sun (around 700,000 km)</u>. The Sun's radius can be extracted from the corresponding SPICE kernel.

Before we go into the programming part we need to answer another question. <i>What is the radius of the Sun?</i> It is somehow connected to the surface, but what is the surface of huge plasma ball.